In [1]:
from IPython.core.display import HTML
css = open('style-table.css').read() + open('style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

In [57]:
import pandas as pd
from pandas import DataFrame, Series

import numpy as np
from collections import namedtuple as tuple

import seaborn as sb

import cPickle as pickle

### FHV (For Hire Vehicle) summary

In [28]:
baseStations = set()
locations = set()

# b2locmap = defaultdict(list)

bases = blist()
locs = blist()
times = blist()

# pkupatbase = heapdict.heapdict() # defaultdict(list)
# pkupatloc = heapdict.heapdict() # defaultdict(list)

## Check performance of different dict usages
## https://biggestfool.tumblr.com/post/21247759480/on-the-speed-of-dictionaries-in-python
## https://writeonly.wordpress.com/2008/08/30/sorting-dictionaries-by-value-in-python-improved/
pkupatbase = {}
pkupatloc = {}

In [46]:
from operator import itemgetter
def sbv6(d,reverse=False):
    ''' proposed in PEP 265, using  the itemgetter '''
    return sorted(d.iteritems(), key=itemgetter(1), reverse=True)

In [ ]:
## Changed csv reading to pandas' read_csv

In [67]:
fhvfile = open('data/taxi/fhv_tripdata_2016-06.csv', 'r')
header = fhvfile.readline().strip().split(',')
fhvfile.close()
print header

['Dispatching_base_num', 'Pickup_date', 'locationID']


In [68]:
fhvdf = pd.read_csv('data/taxi/fhv_tripdata_2016-06.csv', parse_dates=['Pickup_date'])

In [69]:
fhvdf.head()

,Dispatching_base_num,Pickup_date,locationID
0,B00001,2016-06-01 04:00:00,NaN
1,B00001,2016-06-01 04:15:00,NaN
2,B00001,2016-06-01 04:30:00,NaN
3,B00001,2016-06-01 05:00:00,NaN
4,B00001,2016-06-01 05:00:00,NaN


In [39]:
fhvfile = open('data/taxi/fhv_tripdata_2016-06.csv', 'r')
header = fhvfile.readline().strip().split(',')

## Get length of each line for faster string parsing vs. using split()
rowlen = len("B00008,2016-06-04 04:42:00,")

for line in fhvfile:
    ## Each base number is 6 chars long, starting with Bxxxxx, with xxxxx being an integer
    basenum, pkuptime, locid = line.strip().split(',')
    basenum = int(basenum[1:])
    locid = 0 if locid == '' else int(locid)
    bases.append(basenum)
    locs.append(locid)
    times.append(pkuptime)
    baseStations.add(basenum)
    locations.add(locid)
    
    ## count pickups per base station and per location
    ## TODO: speed of creating dictionaries is very slow - consider alternate count methods when concatenating multiple files
    if basenum in pkupatbase:
        pkupatbase[basenum] = pkupatbase[basenum] + 1
    else:
        pkupatbase[basenum] = 1
    
    if locid in pkupatloc:
        pkupatloc[locid] = pkupatloc[locid] + 1
    else:
        pkupatloc[locid] = 1
fhvfile.close()

In [51]:
print header

['Dispatching_base_num', 'Pickup_date', 'locationID']


In [6]:
print "{0} Distinct Base Stations".format(len(baseStations))
print "{0} Distinct Location IDs".format(len(locations))

609 Distinct Base Stations
264 Distinct Location IDs


In [47]:
## How many of the location IDs correspond to each base station? 
## How many pickups occur at each location ID? From each base station?
## How does num pickups vary per base station / location ID with time?

pbase = sbv6(pkupatbase)
ploc = sbv6(pkupatloc)

In [49]:
print pbase[0:10]

[(2764, 1282882), (2510, 1074722), (2682, 1068603), (2867, 560751), (2800, 517964), (2835, 502914), (2836, 320702), (2765, 302154), (2617, 267855), (2864, 217733)]


In [50]:
print ploc[0:10]

[(0, 3471116), (161, 159972), (79, 156218), (234, 141062), (231, 138312), (138, 130485), (170, 123799), (132, 123285), (68, 118689), (230, 117225)]


In [15]:
print locations

set([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 

In [17]:
print baseStations

set([1, 2050, 2731, 2054, 2055, 8, 9, 2058, 2060, 13, 14, 2067, 21, 2073, 29, 30, 2080, 37, 39, 2095, 2096, 2099, 53, 2103, 56, 2106, 2107, 2110, 2111, 2120, 2121, 78, 2061, 2132, 2133, 2138, 95, 2144, 2145, 2147, 2148, 2152, 2157, 112, 2936, 2162, 2163, 2167, 2173, 2175, 131, 2180, 2182, 2191, 2196, 149, 151, 2200, 2207, 160, 2210, 2213, 2217, 170, 171, 2222, 2228, 2229, 183, 2240, 2241, 2245, 2248, 2249, 202, 2423, 2252, 2253, 2254, 208, 2263, 2265, 220, 221, 222, 2271, 225, 227, 2277, 2279, 235, 2285, 2287, 2288, 2292, 2293, 248, 2298, 254, 255, 256, 2306, 2315, 2316, 271, 272, 2435, 277, 280, 281, 290, 2339, 2341, 296, 2346, 2352, 310, 2360, 318, 2101, 325, 2379, 337, 2398, 2389, 2391, 345, 2395, 2397, 350, 2401, 2405, 2406, 2408, 2410, 2416, 369, 2418, 373, 375, 2424, 2426, 2427, 381, 385, 387, 2437, 2440, 401, 2450, 2453, 411, 412, 2461, 2462, 2459, 2468, 2469, 2470, 2471, 2460, 2476, 429, 2480, 2485, 2488, 2491, 445, 446, 448, 449, 2499, 2501, 2503, 457, 459, 2508, 2510, 2511, 2

### Green Taxi Summary

In [71]:
taxifile = open('data/taxi/green_tripdata_2016-06.csv', 'r')
taxiheader = taxifile.readline().strip().split(',')
taxifile.close()
print taxiheader

['VendorID', 'lpep_pickup_datetime', 'Lpep_dropoff_datetime', 'Store_and_fwd_flag', 'RateCodeID', 'Pickup_longitude', 'Pickup_latitude', 'Dropoff_longitude', 'Dropoff_latitude', 'Passenger_count', 'Trip_distance', 'Fare_amount', 'Extra', 'MTA_tax', 'Tip_amount', 'Tolls_amount', 'Ehail_fee', 'improvement_surcharge', 'Total_amount', 'Payment_type', 'Trip_type']


#### Interesting Fields
- VendorID
- trip time = Lpep_dropoff_datetime - lpep_pickup_datetime
- fare_amount
- tip_amount
- total_amount
- payment_type
- trip_type
- ehail_fee

In [64]:
greendf = pd.read_csv("data/taxi/green_tripdata_2016-06.csv", 
                     parse_dates = ["lpep_pickup_datetime", "Lpep_dropoff_datetime"])

In [ ]:
greendf['id'] = greendf.index.values

In [66]:
greendf.head()

,VendorID,lpep_pickup_datetime,Lpep_dropoff_datetime,Store_and_fwd_flag,RateCodeID,Pickup_longitude,Pickup_latitude,Dropoff_longitude,Dropoff_latitude,Passenger_count,...,Fare_amount,Extra,MTA_tax,Tip_amount,Tolls_amount,Ehail_fee,improvement_surcharge,Total_amount,Payment_type,Trip_type
0,2,2016-06-01 02:46:38,2016-06-01 03:06:40,N,1,-73.930580,40.695179,-74.000053,40.729046,1,...,19.5,0.5,0.5,6.24,0.0,NaN,0.3,27.04,1,1.0
1,2,2016-06-01 02:55:26,2016-06-01 03:06:52,N,1,-73.946930,40.792553,-73.951569,40.825161,1,...,11.5,0.5,0.5,2.56,0.0,NaN,0.3,15.36,1,1.0
2,2,2016-06-01 02:50:36,2016-06-01 03:08:39,N,1,-73.944534,40.823956,-73.994659,40.750423,1,...,23.5,0.5,0.5,2.00,0.0,NaN,0.3,26.80,1,1.0
3,2,2016-06-01 02:57:04,2016-06-01 03:07:52,N,1,-73.952209,40.823872,-73.914360,40.814697,1,...,10.5,0.5,0.5,0.00,0.0,NaN,0.3,11.80,2,1.0
4,2,2016-06-01 02:52:03,2016-06-01 03:08:12,N,1,-73.957977,40.717827,-73.954018,40.655121,3,...,16.5,0.5,0.5,0.00,0.0,NaN,0.3,17.80,1,1.0


In [90]:
greendf[['VendorID', 'id']].groupby('VendorID').count()

,id
VendorID,
1,287772
2,1116954


In [72]:
taxifile = open('data/taxi/yellow_tripdata_2016-06.csv', 'r')
taxiheader = taxifile.readline().strip().split(',')
taxifile.close()
print taxiheader

['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance', 'pickup_longitude', 'pickup_latitude', 'RatecodeID', 'store_and_fwd_flag', 'dropoff_longitude', 'dropoff_latitude', 'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount']


In [73]:
yellowdf = pd.read_csv("data/taxi/yellow_tripdata_2016-06.csv", 
                     parse_dates = ["tpep_pickup_datetime", "tpep_dropoff_datetime"])

In [ ]:
yellowdf['id'] = yellowdf.index.values

In [74]:
yellowdf.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2016-06-09 21:06:36,2016-06-09 21:13:08,2,0.79,-73.983360,40.760937,1,N,-73.977463,40.753979,2,6.0,0.5,0.5,0.00,0.0,0.3,7.30
1,2,2016-06-09 21:06:36,2016-06-09 21:35:11,1,5.22,-73.981720,40.736668,1,N,-73.981636,40.670242,1,22.0,0.5,0.5,4.00,0.0,0.3,27.30
2,2,2016-06-09 21:06:36,2016-06-09 21:13:10,1,1.26,-73.994316,40.751072,1,N,-74.004234,40.742168,1,6.5,0.5,0.5,1.56,0.0,0.3,9.36
3,2,2016-06-09 21:06:36,2016-06-09 21:36:10,1,7.39,-73.982361,40.773891,1,N,-73.929466,40.851540,1,26.0,0.5,0.5,1.00,0.0,0.3,28.30
4,2,2016-06-09 21:06:36,2016-06-09 21:23:23,1,3.10,-73.987106,40.733173,1,N,-73.985909,40.766445,1,13.5,0.5,0.5,2.96,0.0,0.3,17.76


In [93]:
yellowdf[['id', 'passenger_count']].groupby(by = ['passenger_count']).count()

,id
passenger_count,
0,406
1,7898295
2,1608298
3,468831
4,224308
5,576247
6,359006
7,31
8,27


In [94]:
yellowdf[['id', 'VendorID']].groupby(by = ['VendorID']).count()

,id
VendorID,
1,5235710
2,5899760


#### Questions for taxi data
 - How can latitude and longitude be mapped to a location / borough as contained in the taxi_zone_lookup file?
 - Does it make sense to have passenger counts of 7-9 people? How would 6 fit? (mini van??)
 - How does this data look like bucketed by 6 hour time intervals (12am-6am, 6am-12pm, 12pm-6pm, 6pm-12am)?